[Sentiment Analysis: Beyond Words](https://towardsdatascience.com/sentiment-analysis-beyond-words-6ca17a6c1b54)

In [0]:
from IPython.display import clear_output
# Import necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

path = "/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/InsideAirbnb/data/"
calendar = pd.read_csv(path+"calendar.csv")
listings = pd.read_csv(path+"listings.csv")
reviews = pd.read_csv(path+"reviews.csv")
clear_output()

# Column lists

In [0]:
col_list = list(listings.columns)
# basicly the text information about the host
text_columns=['summary','space','description',"neighborhood_overview","notes","transit","access","interaction","house_rules","host_about"]
# 
host_string_columns = ["host_name","host_since","amenities"]

host_location_columns = ["host_neighbourhood","street","neighbourhood","neighbourhood_cleansed","city","market","host_location",
                         "latitude","longitude"]

host_numeric_columns = ["host_id","host_acceptance_rate","host_response_time","host_response_rate","host_listings_count",
                        "host_total_listings_count","square_feet","price","weekly_price","monthly_price",
                        "security_deposit","cleaning_fee","guests_included","extra_people","number_of_reviews",
                        "number_of_reviews_ltm","calculated_host_listings_count","calculated_host_listings_count_entire_homes",
                        "calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms",
                        "reviews_per_month"]

host_date_columns = ["first_review"]

host_nights_columns = col_list[67:75]

host_category_columns = ["host_is_superhost","host_has_profile_pic","host_identity_verified","property_type","room_type",
                         "instant_bookable","is_business_travel_ready","require_guest_profile_picture","require_guest_phone_verification"]

host_score_columns = ["review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin",
                      "review_scores_communication","review_scores_location","review_scores_value"
                      ]
host_other_columns = ["host_verifications",]

# 1. Sentiment scoring with VADER

In [0]:
# load the SentimentIntensityAnalyser object in
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()
def print_sentiment_scores(sentence):
    snt = analyzer.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(snt)))
clear_output()

In [0]:
# getting only the negative score
def negative_score(text):
    negative_value = analyzer.polarity_scores(text)['neg']
    return negative_value

# getting only the neutral score
def neutral_score(text):
    neutral_value = analyzer.polarity_scores(text)['neu']
    return neutral_value

# getting only the positive score
def positive_score(text):
    positive_value = analyzer.polarity_scores(text)['pos']
    return positive_value

# getting only the compound score
def compound_score(text):
    compound_value = analyzer.polarity_scores(text)['compound']
    return compound_value

In [15]:
target = text_columns[0]
df_text = listings[['id']+text_columns]
df = df_text[[target]]
df.fillna('',inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [0]:
def get_sentiment_score_vader(df_eng,target):
  df_eng[target+'_sentiment_neg'] = df_eng[target].apply(negative_score)
  df_eng[target+'_sentiment_neu'] = df_eng[target].apply(neutral_score)
  df_eng[target+'_sentiment_pos'] = df_eng[target].apply(positive_score)
  df_eng[target+'_sentiment_compound'] = df_eng[target].apply(compound_score)
  return df_eng

In [0]:
df_sentiment_summary = get_sentiment_score_vader(df,target)
clear_output()

In [23]:
df_sentiment_summary.head()

,summary,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound,summary_sentiment_neg,summary_sentiment_neu,summary_sentiment_pos,summary_sentiment_compound
0,Our house was built in 1937 when there was ple...,0.000,0.870,0.130,0.9167,0.000,0.870,0.130,0.9167
1,,0.000,0.000,0.000,0.0000,0.000,0.000,0.000,0.0000
2,Our Cottage is a charming light filled cottage...,0.017,0.876,0.107,0.8658,0.017,0.876,0.107,0.8658
3,Welcoming victorian house. Breakfast included....,0.000,0.807,0.193,0.8439,0.000,0.807,0.193,0.8439
4,washing can be done at a cost of five euro per...,0.000,0.957,0.043,0.0772,0.000,0.957,0.043,0.0772


In [28]:
df_sentiment_combined = pd.DataFrame()
df_sentiment_combined['id'] = listings['id']
for target in text_columns:
  df = listings[[target]]
  df.fillna('',inplace=True)
  df_sentiment = get_sentiment_score_vader(df,target)
  df_sentiment_combined = pd.concat([df_sentiment_combined,df_sentiment],axis=1)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [0]:
df_sentiment_combined.to_csv('/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/DataMining_Assignment/Data/df_sentiment_combined.csv')

In [31]:
df_sentiment_combined.head()

,id,summary,summary_sentiment_neg,summary_sentiment_neu,summary_sentiment_pos,summary_sentiment_compound,space,space_sentiment_neg,space_sentiment_neu,space_sentiment_pos,space_sentiment_compound,description,description_sentiment_neg,description_sentiment_neu,description_sentiment_pos,description_sentiment_compound,neighborhood_overview,neighborhood_overview_sentiment_neg,neighborhood_overview_sentiment_neu,neighborhood_overview_sentiment_pos,neighborhood_overview_sentiment_compound,notes,notes_sentiment_neg,notes_sentiment_neu,notes_sentiment_pos,notes_sentiment_compound,transit,transit_sentiment_neg,transit_sentiment_neu,transit_sentiment_pos,transit_sentiment_compound,access,access_sentiment_neg,access_sentiment_neu,access_sentiment_pos,access_sentiment_compound,interaction,interaction_sentiment_neg,interaction_sentiment_neu,interaction_sentiment_pos,interaction_sentiment_compound,house_rules,house_rules_sentiment_neg,house_rules_sentiment_neu,house_rules_sentiment_pos,house_rules_sentiment_compound,host_about,host_about_sentiment_neg,host_about_sentiment_neu,host_about_sentiment_pos,host_about_sentiment_compound
0,44077,Our house was built in 1937 when there was ple...,0.000,0.870,0.130,0.9167,"Hi, My name is Teresa and we have 3 rooms ava...",0.009,0.832,0.159,0.9826,Our house was built in 1937 when there was ple...,0.000,0.854,0.146,0.9808,I like our neighbourhood as there is no shorta...,0.061,0.809,0.130,0.7003,Dundrum was always known as a Spa Town and the...,0.000,0.755,0.245,0.9136,There is a bus stop 2 mins up the road from us...,0.041,0.959,0.000,-0.2960,We have a beautiful landscaped garden with tab...,0.000,0.831,0.169,0.8750,We live on the premises so we are always on ha...,0.0,0.831,0.169,0.6249,Welcome to our cottage and we hope you will en...,0.000,0.838,0.162,0.9747,"I am a friendly outgoing Irishwoman ,who love...",0.000,0.584,0.416,0.9786
1,85148,,0.000,0.000,0.000,0.0000,I'm renting a double room in my house in the s...,0.023,0.886,0.091,0.9231,I'm renting a double room in my house in the s...,0.023,0.886,0.091,0.9231,,0.000,0.000,0.000,0.0000,,0.000,0.000,0.000,0.0000,,0.000,0.000,0.000,0.0000,,0.000,0.000,0.000,0.0000,,0.0,0.000,0.000,0.0000,"Sorry, No smoking or pets. Lo siento esta proh...",0.363,0.637,0.000,-0.5719,I am single and live alone here so when I'm ...,0.067,0.774,0.159,0.9499
2,85156,Our Cottage is a charming light filled cottage...,0.017,0.876,0.107,0.8658,"Hi , My name is Teresa and we have 3 rooms av...",0.000,0.927,0.073,0.9214,Our Cottage is a charming light filled cottage...,0.009,0.896,0.096,0.9544,"I love Dundrum and its surrounding areas , The...",0.011,0.777,0.212,0.9906,We are always prepared to go the extra mile so...,0.000,0.688,0.312,0.8957,There is a tram service to the city about 10 m...,0.000,0.941,0.059,0.9107,"Guests have access to the kitchen to make tea,...",0.000,0.815,0.185,0.7783,We live on site so we are available to help yo...,0.0,0.717,0.283,0.9300,Welcome to our cottage and we hope you will en...,0.000,0.832,0.168,0.9756,"I am a friendly outgoing Irishwoman ,who love...",0.000,0.584,0.416,0.9786
3,121030,Welcoming victorian house. Breakfast included....,0.000,0.807,0.193,0.8439,"Double room 1 upstairs,wooden floors, desk, in...",0.011,0.869,0.120,0.9660,Welcoming victorian house. Breakfast included....,0.000,0.856,0.144,0.9778,"My neighbourhood is very safe, friendly and o...",0.000,0.719,0.281,0.9454,I can help you with questions about history an...,0.000,0.906,0.094,0.4019,"The bus is at the corner, the dart train is te...",0.000,1.000,0.000,0.0000,"I have two bathrooms, one upstairs one downsta...",0.000,0.898,0.102,0.3612,I will give you suggestions for food music and...,0.0,1.000,0.000,0.0000,"Please don't smoke , safety and I hate the sme...",0.107,0.717,0.176,0.7370,I am a well traveled person with lots of cultu...,0.000,0.715,0.285,0.9937
4,159889,washing can be done at a cost of five euro per...,0.000,0.957,0.043,0.0772,Nice comfortable house 20 minute

# Apply on Reviews

In [0]:
!pip install langdetect
from langdetect import detect
clear_output()

In [0]:
def language_detection(text):
  try:
    return detect(text)
  except:
    print(text)
    return None

In [53]:
(reviews.shape[0]/5000) *22/60

25.17526

In [0]:
df_sample = reviews#.sample(5000)
df_sample['language'] = df_sample['comments'].apply(language_detection)
df_sample.head(3)

In [0]:
df_sample.to_csv("/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/DataMining_Assignment/Data/reviews_language_detection.csv")

In [58]:
df = df_sample[df_sample['language']=='en']
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,language
0,44077,203703,2011-03-20,393348,Christian,We enjoyed our stay very much. The room was co...,en
1,44077,211369,2011-03-28,444004,Solidea,We have been here 4 nights. Stay in a home is ...,en
2,44077,234215,2011-04-21,465058,Michael And Isabelle,Teresa and Hughie were great hosts. They were ...,en
3,44077,261843,2011-05-13,490005,Weston,"No surprises, was as described. Very gracious...",en
4,44077,268148,2011-05-17,520460,Barbara,"Teresa was a lovely hostess, and we had a deli...",en


In [0]:
target = 'comments'
df = df[[target]]
df = df.fillna('')

In [0]:
df_sentiment = get_sentiment_score_vader(df,target)

In [61]:
df_sentiment['id'] = df_sample['id']
df_sentiment['listing_id'] = df_sample['listing_id']
df_sentiment.head()

,comments,comments_sentiment_neg,comments_sentiment_neu,comments_sentiment_pos,comments_sentiment_compound,id,listing_id
0,We enjoyed our stay very much. The room was co...,0.108,0.603,0.289,0.9513,203703,44077
1,We have been here 4 nights. Stay in a home is ...,0.000,0.558,0.442,0.9850,211369,44077
2,Teresa and Hughie were great hosts. They were ...,0.000,0.736,0.264,0.9836,234215,44077
3,"No surprises, was as described. Very gracious...",0.048,0.549,0.403,0.9447,261843,44077
4,"Teresa was a lovely hostess, and we had a deli...",0.000,0.588,0.412,0.9835,268148,44077


In [0]:
df_sentiment.to_csv("/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/DataMining_Assignment/Data/reviews_sentiments.csv")

# 2. Sentiment with Stanford Core NLP
Need run local, I will do it later if we have time

In [0]:
!pip install pycorenlp
from pycorenlp import StanfordCoreNLP
clear_output()

In [37]:
!cd stanford-corenlp-full-2018-10-05
!java -mx5g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 10000

/bin/bash: line 0: cd: stanford-corenlp-full-2018-10-05: No such file or directory
Error: Could not find or load main class edu.stanford.nlp.pipeline.StanfordCoreNLPServer
Caused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLPServer


In [0]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [0]:
def get_sentiment(text):
    res = nlp.annotate(text,
                       properties={'annotators': 'sentiment',
                                   'outputFormat': 'json',
                                   'timeout': 1000,
                       })
    print(text)
    print('Sentiment:', res['sentences'][0]['sentiment'])
    print('Sentiment score:', res['sentences'][0]['sentimentValue'])
    print('Sentiment distribution (0-v. negative, 5-v. positive:', res['sentences'][0]['sentimentDistribution'])

In [40]:
text = 'Passing in the review of the food as garbage, the model classifies the overall sentence as pretty negative'
get_sentiment(text=text)

Exception: ignored